In [1]:
import numpy as np
import random as rd
import asyncio
from collections import deque
import math
import random as rd

# PREPARATION BLOCK

In [2]:
file = open('start_swaptable','r')
start_swaptable = []
for i in file:
    start_swaptable.append(int(i)-1)
    
file = open('final_swaptable','r')
final_swaptable = []
for i in file:
    final_swaptable.append(int(i)-1)
    
file = open('f_final_swaptable','r')
f_final_swaptable = []
for i in file:
    f_final_swaptable.append(int(i)-1)

file = open('escalC','r')
escalC = []
for i in file:
    escalC.append(int(i)-1)

file = open('escalD','r')
escalD = []
for i in file:
    escalD.append(int(i)-1)
    
file = open('CP','r')
CP = []
for i in file:
    CP.append(int(i)-1)
    
S = []
for i in range(8):
    file = open('S'+str(i)+'','r')
    S.append(np.arange(64))
    k = 0
    for j in file:
        S[i][k] = int(j)
        k += 1
    S[i] = S[i].reshape((4, 16))

# GENARATOR KEY

In [3]:
class Generator_key(object):
    
    def __init__(self,n = 80,m = 140):
        if n <= 2 and m <= n:
            self.n = n
            self.m = m
            print("Encorrect data! Default: n = 80, m = 140")
        self.n = n
        self.m = m
    
    @staticmethod
    def random_sample(count, start, stop, step=1):
        def gen_random():
            while True:
                yield rd.randrange(start, stop, step)
        def gen_n_unique(source, n):
            seen = set()
            seenadd = seen.add
            for i in (i for i in source() if i not in seen and not seenadd(i)): 
                yield i 
                if len(seen) == n:
                    break
        return [i for i in gen_n_unique(gen_random, min(count, int(abs(stop - start) / abs(step))))] 
    
    @staticmethod
    def isPrime(n):
        if n % 2 == 0 or n < 0:
            return n == 2
        d = 3
        while d * d <= n and n % d != 0:
            d += 2
        return d * d > n 
    
    @staticmethod
    def MillerRabinTest(n,k):
        if n < 10000:
            print("Use function isPrime!")
            
        if n % 2 == 0:
            return print(n," - not a prime.")
        
        t = n - 1
        s = 0
        
        while t % 2 == 0:
            t //= 2
            s += 1
            
        list_a  =  list(map(int, Generator_key.random_sample(1000, 2, n - 1)))
        
        for i in range(k):
            a = rd.choice(list_a)
            list_a.remove(a)
            x = pow(a, t, n)
            if x == 1 or x == n - 1:
                continue
            for r in range(1,s):
                x = pow(x, 2, n)
                if (x == 1):
                    return print(n," - not a prime.")
                if (x == n - 1):
                    break
                    
            if (x != n - 1):
                return print(n," - not a prime.")
            
        return print(n," - prime. k = ",0.25**k," - the probability that it can be a composite number.")
        
    def __start_q(self):
        q = rd.choice(range(self.n,self.m))
        while self.isPrime(q) != True:
            q = rd.choice(range(self.n,self.m))
        return q
    
    def __r_return(self,q):
        r =  rd.choice(list(map(int, self.random_sample(1000, q, 4*q + 3))))
        while r % 2 != 0:
            r =  rd.choice(list(map(int, self.random_sample(1000, q, 4*q + 3))))
        return r 
 
    def __n_primary(self,q0,n0):            
        q =  q0
        n = n0
        attempts = 0
        a  =  rd.choice(list(map(int, self.random_sample(1000, 2, n - 1))))
        while pow(a,n-1,n) != 1 or pow(a,(n-1)//q,n) == 1:
            if attempts % 100 == 0:
                r = self.__r_return(q)
                n = q*r + 1
            a  =  rd.choice(list(map(int, self.random_sample(1000, 2, n - 1))))
            attempts += 1 
        return n
    
    def buildBIGPrimeNumber64(self):
        q = self.__start_q()
        while len(bin(q))-1 != 64:
            q = self.__start_q()
            for i in range(3):
                r = self.__r_return(q)
                n = q*r + 1
                q = self.__n_primary(q,n) 
        return q
    

# RDES

In [4]:
class RDES(object):
    
    @staticmethod
    def fix_bit(number, bits = 16):
        return format(number & 2 ** bits - 1, "0{}b".format(bits))
    
    @staticmethod
    def toBits(text,lbits=16): 
        text_bits = "".join(RDES.fix_bit(ord(i),lbits) for i in text)    
        if len(text_bits) % 64 != 0:
            if len(text_bits) < 64:
                text_bits = "".join(str(0) for i in range(64 - len(text_bits))) + text_bits
            elif len(text_bits) > 64:
                text_bits = "".join(str(0) for i in range(64 - len(text_bits)%64)) + text_bits            
        return text_bits
    
    @staticmethod
    def get_blocks_bits(text_bits):
        blocks_bits = []
        for i in range(0,len(text_bits),64):
            blocks_bits.append("".join(text_bits[i:i+64]))
        return blocks_bits
                
    @staticmethod
    def toDigit64(bits):
        return [int(bits[i:i+16],2) for i in range(0,len(bits),16)]
    
    @staticmethod
    def toSigns(bits,lbits=16):
        return "".join(chr(int(bits[i:i+lbits],2)) for i in range(0,len(bits),lbits))
    
    @staticmethod
    def XOR(a,b):
        if len(a) != len(b):
            print("error! len(a) != len(b)")
        else:
            c = np.array([int(i) for i in a]) ^ np.array([int(i) for i in b])
            c = list(c)
        return "".join(str(i) for i in c)
    
    @staticmethod
    def createKey(): 
        key64 = Generator_key().buildBIGPrimeNumber64()
        Generator_key.MillerRabinTest(key64,100)
        
        key64 = RDES.fix_bit(key64,64)
        extra_random_indexs = [rd.choice(np.arange(64)) for i in range(16)]
        
        key128 = "".join(RDES.fix_bit(i,16) for i in extra_random_indexs)    
        key192 = key128 + key64
        
        return (RDES.toSigns(key192))
    
    def __escalation_key(self,key64): 
        array_keys = []
        C = "".join(str(i) for i in [key64[j] for j in escalC])
        D = "".join(str(i) for i in [key64[j] for j in escalD])
        for i in range(16):
            if i == 0 or i == 2 or i == 8 or i == 15:
                c = deque(list(C))
                c.rotate(-2)
                c = list(c)
                C = "".join(str(i) for i in [j for j in c])
                
                d = deque(list(D))
                d.rotate(-2)
                d = list(d)
                D = "".join(str(i) for i in [j for j in d])
                
                array_keys.append(self.__swap_array56(C + D,CP))
                
            else:
                c = deque(list(C))
                c.rotate(-1)
                c = list(c)
                C = "".join(str(i) for i in [j for j in c])
                
                d = deque(list(D))
                d.rotate(-1)
                d = list(d)
                D = "".join(str(i) for i in [j for j in d])
                
                array_keys.append(self.__swap_array56(C + D,CP))
                
        return array_keys
    
    
    
    
    def __swap_array64(self,ar_initial64, ar_rule):
        return "".join(str(i) for i in [ar_initial64[i] for i in ar_rule])
    
    def __swap_array56(self,ar_initial56,ar_rule):
        return "".join(str(i) for i in [ar_initial56[i] for i in ar_rule])
    
    def __swap_array32(self,ar_initial32, ar_rule):
        return "".join(str(i) for i in [ar_initial32[i] for i in ar_rule])
    
    def __EP(self,input_quer32):
        l = 1
        output_quer48 = 48*[0]
        output_quer48[0] = input_quer32[31]
        output_quer48[1] = input_quer32[0]
        output_quer48[47] = input_quer32[0]
        output_quer48[46] = input_quer32[31]
        for i in range(2,45,5):
            output_quer48[i] = input_quer32[l]
            output_quer48[i+1] = input_quer32[l+1]
            output_quer48[i+2]= output_quer48[i+4] = input_quer32[l+2]
            output_quer48[i+3]= output_quer48[i+5] = input_quer32[l+3]
        return "".join(str(i) for i in output_quer48)
    
    
  
    def __f_encryp(self,B032,round_key48):
        arr_choice = 8*[0]
        
        output_quer48 = self.__EP(B032)
        xor_sum = RDES.XOR(output_quer48,round_key48)
        
        block_bits = np.array([int(i) for i in xor_sum]).reshape((8,6))
        
        for i in range(8):
            arr_choice[i] = S[i][int(''.join(str(i) for i in block_bits[i][:2]),2)][int(''.join(str(i) for i in block_bits[i][2:6]),2)]
        
        pre_output_bits = "".join(RDES.fix_bit(i,4) for i in arr_choice)
        return self.__swap_array32(pre_output_bits,f_final_swaptable)
    
    
    
    def __init(self,array64,key192):
        return self.__swap_array64(array64,start_swaptable), self.__escalation_key(RDES.toBits(key192)[256:320]) 
    
    def __enc_DES_algorithm(self,array64,key192):
        array64 ,array_keys = self.__init(array64,key192)[0], self.__init(array64,key192)[1]
        A0 = array64[0:32]
        B0 = array64[32:]
        for i in range(16):
            if (i == 15):
                A0 = RDES.XOR(A0, self.__f_encryp(B0,array_keys[i]))
                break
            if int(RDES.toBits(key192)[256:320][RDES.toDigit64(RDES.toBits(key192[:256]))[i]]) == 1:
                register = B0
                B0 = RDES.XOR(A0, self.__f_encryp(B0,array_keys[i]))
                A0 = register
            else:
                A0 = RDES.XOR(A0, self.__f_encryp(B0,array_keys[i])) 
        return self.__swap_array64(A0 + B0, final_swaptable)
        
        
    def __decr_DES_algorithm(self,array64,key192):
        array64 ,array_keys = self.__init(array64,key192)[0], self.__init(array64,key192)[1]
        A0 = array64[0:32]
        B0 = array64[32:]
        for i in range(15,-1,-1):
            if (i == 0):
                A0 = RDES.XOR(A0, self.__f_encryp(B0,array_keys[i]))
                break
            if int(RDES.toBits(key192)[256:320][RDES.toDigit64(RDES.toBits(key192[:256]))[i-1]]) == 1:
                register = B0
                B0 = RDES.XOR(A0, self.__f_encryp(B0,array_keys[i]))
                A0 = register
            else:
                A0 = RDES.XOR(A0, self.__f_encryp(B0,array_keys[i]))
        return self.__swap_array64(A0 + B0, final_swaptable)
        
    def DES_encr(self,text,key192):
        blocks_bits = RDES.get_blocks_bits(RDES.toBits(text))
        encr_blocks_bits = []
        for block in blocks_bits:
            encr_blocks_bits.append(self.__enc_DES_algorithm(block,key192))
        encr_text = "".join(i for i in encr_blocks_bits)
        return RDES.toSigns(encr_text,8)
    
    def DES_decr(self,encr_text,key192):
        encr_blocks = RDES.get_blocks_bits(RDES.toBits(encr_text,8))
        decr_blocks_bits = []
        for block in encr_blocks:
            decr_blocks_bits.append(self.__decr_DES_algorithm(block,key192))
        decr_text = "".join(i for i in decr_blocks_bits)
        return RDES.toSigns(decr_text,16)

# TEST BLOCK

In [5]:
key = RDES.createKey()

7413997077740221613  - prime. k =  6.223015277861142e-61  - the probability that it can be a composite number.


In [6]:
text1 = "Cryptography — the science of secret writing — is an ancient art; the first documented use of cryptography in writing dates back to circa 1900 B.C. when an Egyptian scribe used non-standard hieroglyphs in an inscription. Some experts argue that cryptography appeared spontaneously sometime after writing was invented, with applications ranging from diplomatic missives to war-time battle plans. It is no surprise, then, that new forms of cryptography came soon after the widespread development of computer communications. In data and telecommunications, cryptography is necessary when communicating over any untrusted medium, which includes just about any network, particularly the Internet."
text2 = "Криптография — это наука, изучающая способы сокрытия данных и обеспечения их конфиденциальности. Это одна из старейших наук и ее история насчитывает четыре тысячелетия. Сам термин “криптография” образовался от двух древнегреческих слов “крипто” — скрытый, “графо” — пишу. Для начинающих принцип криптографии можно объяснить на примере шифра Цезаря, где каждый символ алфавита был заменен на тот, который находится на 3 позиции до нужного."

In [7]:
print("key: " + key + "\n")
print("text: ",text2 , "\n")
encr_text = RDES().DES_encr(text2,key)
print("Encrypted text: ", encr_text, "\n")
decr_text = RDES().DES_decr(encr_text,key)
print("Decrypted text: ", decr_text)

key: =)	6,)7>=,.-曣켏꡽璭

text:  Криптография — это наука, изучающая способы сокрытия данных и обеспечения их конфиденциальности. Это одна из старейших наук и ее история насчитывает четыре тысячелетия. Сам термин “криптография” образовался от двух древнегреческих слов “крипто” — скрытый, “графо” — пишу. Для начинающих принцип криптографии можно объяснить на примере шифра Цезаря, где каждый символ алфавита был заменен на тот, который находится на 3 позиции до нужного. 

ÄnÜ¬uÀØR:¸Ë%¶R¡2 Ø%R*÷=«ÄjÜ¬uÀu%F*úÝúZ8:¸Ë%°q¨­ Z_Z"J%R2û-q­«ZTZ%ñ*öÝûq§¥ZUZ ÌCÜ¬uÀ×q¯¥ZUZ"%ð*ûÝú/ÌJÜ¡}©ÀÝR2 Ù% R®:¸ Ó-R2 Ý%q­ªZ\Z%q­¤ZXRÌJÜ¤}©ÀÐq­®Z_Z+R2 Ù%R2 Ð%q­­ZUZR:÷%R¥2÷%¾R2 ×-R2÷%q©­ ZWZ%ÌLÜªuÀZ12 ×%R:¸ Ó-R2÷%·ÌøÜ¯uÀuÌJÜ¡}©ÀÛ%ñ*ùÝöÌÿÜ©uÀÕÌ@Ü¯}©À~%ÿ*úÝú.q¥ZVZ%C*ôÝöq­ªZVZ+R«:÷%R2÷%ÌDÜ®uÀÑR:¸Ë%¶R¡2 Ø%q­£ZWZ/q ­¤ZXB1Z12 Õ%R2 Ú%²q­§ZUZy8­­ZVRq­¡ZTZy8­¨ZWZ%R2 Ý%°q¨­ªZ\Z%R£2 Ó%
:¸Ï-q­¤ZSZZ9:¸ Ï%´R«:¸ Ù%³R¯2 Ù%³Ì÷Ü¨uÀy8­¯ZWZ+Ì@Ü§